In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
from fastai.plots import *
from fastai.imports import *
from fastai.transforms import *
from fastai.dataset import *

from sklearn.metrics import fbeta_score
import warnings

ModuleNotFoundError: No module named 'fastai'

In [16]:
plt.ion()

In [17]:
PATH = '/mnt/samsung_1tb/Data/fastai/planet/'

In [18]:
def get_1st(path):
    return glob(f'{path}/*.*')[0]

In [19]:
list_paths = [f'{PATH}train-jpg/train_0.jpg', f'{PATH}train-jpg/train_1.jpg']
titles = ['hazt primary', 'agriculture clear primary water']

In [20]:
#from planet.py
def f2(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([fbeta_score(targs, (preds>th), 2, average='samples')
                    for th in np.arange(start,end,step)])

def opt_th(preds, targs, start=0.17, end=0.24, step=0.01):
    ths = np.arange(start,end,step)
    idx = np.argmax([fbeta_score(targs, (preds>th), 2, average='samples')
                for th in ths])
    return ths[idx]

def get_data(path, tfms, bs, n, cv_idx):
    val_idxs = get_cv_idxs(n, cv_idx)
    return ImageClassifierData.from_csv(path, 'train-jpg', f'{path}train_v2.csv', bs, tfms,
                                 suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

def get_data_zoom(f_model, path, sz, bs, n, cv_idx):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return get_data(path, tfms, bs, n, cv_idx)

def get_data_pad(f_model, path, sz, bs, n, cv_idx):
    transforms_pt = [RandomRotateZoom(9, 0.18, 0.1), RandomLighting(0.05, 0.1), RandomDihedral()]
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_pt, pad=sz//12)
    return get_data(path, tfms, bs, n, cv_idx)


In [21]:
metrics = [f2]
f_model = resnet34

In [22]:
label_csv = f'{PATH}train_v2.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [23]:
def local_get_data(sz, bs=64):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    data = ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, bs=bs, tfms=tfms, suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')
    return data

In [24]:
data = local_get_data(sz=256)

In [25]:
x,y = next(iter(data.val_dl))

In [26]:
learn = ConvLearner.pretrained(f_model,data,metrics=metrics)

In [27]:
lr = 0.2

In [28]:
lrs = np.array([lr/9, lr/3, lr])

In [29]:
sz=256

In [30]:
#drop batch size to fix cude out of memory error
bs = 48

In [32]:

learn.set_data(local_get_data(sz, bs))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

<p>Failed to display Jupyter Widget of type <code>HBox</code>.</p>
<p>
  If you're reading this message in the Jupyter Notebook or JupyterLab Notebook, it may mean
  that the widgets JavaScript is still loading. If this message persists, it
  likely means that the widgets JavaScript library is either not installed or
  not enabled. See the <a href="https://ipywidgets.readthedocs.io/en/stable/user_install.html">Jupyter
  Widgets Documentation</a> for setup instructions.
</p>
<p>
  If you're reading this message in another frontend (for example, a static
  rendering on GitHub or <a href="https://nbviewer.jupyter.org/">NBViewer</a>),
  it may mean that your frontend doesn't currently support widgets.
</p>

epoch      trn_loss   val_loss   f2         
    0      0.112613   0.103461   0.912557  
    1      0.111936   0.103405   0.913348  
    2      0.112666   0.102604   0.913545  
    3      0.115726   0.104146   0.911876  
    4      0.109607   0.102239   0.914558  
    5      0.111972   0.101516   0.915186  


In [33]:
learn.unfreeze()
#cuda out of memory error on 1080ti using sz=256, bs=64
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

<p>Failed to display Jupyter Widget of type <code>HBox</code>.</p>
<p>
  If you're reading this message in the Jupyter Notebook or JupyterLab Notebook, it may mean
  that the widgets JavaScript is still loading. If this message persists, it
  likely means that the widgets JavaScript library is either not installed or
  not enabled. See the <a href="https://ipywidgets.readthedocs.io/en/stable/user_install.html">Jupyter
  Widgets Documentation</a> for setup instructions.
</p>
<p>
  If you're reading this message in another frontend (for example, a static
  rendering on GitHub or <a href="https://nbviewer.jupyter.org/">NBViewer</a>),
  it may mean that your frontend doesn't currently support widgets.
</p>

epoch      trn_loss   val_loss   f2         
    0      0.099218   0.089878   0.926948  
    1      0.097697   0.088877   0.927181  
    2      0.091735   0.085768   0.929006  
    3      0.096694   0.088886   0.92608   
    4      0.091575   0.084826   0.930594  
    5      0.087652   0.083322   0.931331  
 21%|██        | 143/675 [00:29<01:51,  4.78it/s, loss=0.0846]

In [36]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)
f2(preds, y)

0.93010855108915

In [37]:
val = learn.predict()

In [38]:
f2(val, data.val_y)

0.9282392746903552

In [40]:
f2(learn.TTA(), data.val_y)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
f2(val, data.val_y)

In [ ]:
f2(learn.TTA(), data.val_y)

In [ ]:
def get_labels(a): return [data.classes[o] for o in a.nonzero()[0]]
lbls = test>0.2
idx=print(get_labels(lbls[idx]))
PIL.Image.open(path+data.test_dl.dataset.fnames[idx]).convert('RGB')


In [ ]:
res = [get_labels(o) for o in lbls]
data.test_dl.dataset.fnames[:5]

In [ ]:
outp = pd.DataFrame({'image_name': [f[9:-4] for f in data.test_dl.dataset.fnames], 'tags': [' '.join(l) for l in res]})
outp.head()

In [ ]:
outp.to_csv('tmp/subm.gz', compression='gzip', index=None)